In this Notebook, we explore an implementation of the Two Player Graph Coloring game using our existing codebase.

Naturally, we begin by importing the necessary files

In [24]:
import sys
sys.path.append("../..")

from generic_classes.player import Player
from generic_classes.board import Board
from generic_classes.referee import Referee
from generic_classes.game import Game
from generic_classes.ruleset_interface import RulesetInterface
from generic_classes.strategy_interface import StrategyInterface

from random import randrange

We now need to implement a ruleset for the game.

In [92]:
class GraphColoringRuleset(RulesetInterface):

    def __init__(self, name, initial_state, bounds = None):
        """
        Rulset constructor.

        Args:
            name : The name of the ruleset
            initial_state : The initial state of the board
            bounds : (Optional) The bounds of the board
        """
        self.name = name
        self.initial_state = initial_state
        self.bounds = bounds

    def is_legal(self, board, proposed_move):
        """
        Determines if the move proposed is legal.

        Args:
            board : The board being played on
            proposed_move : The move being proposed

        Return:
            True if the move was legal, else False.
        """
        # If the vertex was not uncolored, the move is invalid
        if board.state[proposed_move["vertex"]]["color"] != 0:
            return False
        
        # Otherwise, iterate over the neighbors of the vertex
        for neighbor in board.state[proposed_move["vertex"]]["adj"]:
            # If any of the neighbors are colored with the same color, the move is invalid
            if board.state[neighbor]["color"] == proposed_move["color"]:
                return False
            
        return True

    def is_game_over(self, board):
        """
        Determines if the game is over.

        Args:
            board : The board being played on

        Return:
            True if the state of the board is an end-game state, else False.
        """
        # Case 1: Player 1 wins, the graph is entirely colored
        num_colored = 0
        for vtx in range(len(board.state)):
            if board.state[vtx]["color"] != 0:
                num_colored += 1
        
        # If the numbebr of nodes colored is equal to the number of nodes in the graph, the game is over
        if num_colored == len(board.state):
            return True
        
        # Case 2: The graph is in a state in which it is impossible to color any more vertices
        for vtx in range(len(board.state)):
            
            # If there exists an uncolored vertex
            if board.state[vtx]["color"] == 0:
                # Create a set of the neighbor's colors
                neighboring_colors = set()
                
                # Populate the set with the available colors
                for neighbor in board.state[vtx]["adj"]:
                    neighboring_colors.add(board.state[neighbor]["color"])
                
                # Special case: Remove "uncolored" from the neighboring colors set
                if 0 in neighboring_colors:
                    neighboring_colors.remove(0)
                
                # If the number of neighboring colors is equal to the bounds of the board, there are no available colors left to choose from
                if len(neighboring_colors) == board.bounds:
                    return True
        return False
                
                

    def update_board(self, board, player, move):
        """
        Update the board based on the player's given move.
        Also updates the board's `data` field to log move history.

        Args:
            board : The board being played on
            player : The player who made the move
            move : The move being made
        """
        board.data[str(board.state)] = (player.name, move)
        
        board.state[move["vertex"]]["color"] = move["color"]

Now to implement a very primitive strategy.

In [120]:
class GraphColoringStrategy(StrategyInterface):

    def __init__(self, name, data = None):
        """
        Strategy constructor.

        Args:
            name : The name of the strategy
            data : (Optional) Data to read from to influence moves.
        """
        self.name = name
        self.data = data

    def move(self, board):
        """
        Make a move based on colors available.
        This is psuedo-smart in the sense that it attempts to choose the lowest-index
        vertex and colors it with the lowest-value available color in the lost.

        Args:
            board : The board being played on

        Return:
            A proposed move.
        """
        #vertex = randrange(0, len(board.state))
        #color = randrange(1, board.bounds + 1)
        for vtx in range(len(board.state)):
            if board.state[vtx]["color"] == 0:
                for color in range(1, board.bounds + 1):
                    for neighbor in board.state[vtx]["adj"]:
                        if board.state[neighbor]["color"] == color:
                            break
                        return {"vertex": vtx, "color": color}
        
        #return {"vertex": vertex, "color": color}
        return None

The board state will be the graph itself, with the initial state being an uncolored graph
* The graph is list of dictionaries
* Each dictionary is a vertex, with the index in the list being its ID
* Each node has a "color" field, representing what color it is, and an "adj" field, representing its neighbors
* The "adj" field is a list of IDs, with each ID being the ID of a neighboring vertex
* Each node's default color is 0

The bounds of the graph will be the maximum number of colors that can be used

A `move` is a dictionary in the form of `{"vertex": int, "color": int}`

In [105]:
initial_state = [
    {"color": 0, "adj": [1, 3]},
    {"color": 0, "adj": [0, 2, 3]},
    {"color": 0, "adj": [1]},
    {"color": 0, "adj": [0, 1]},
]

In [106]:
bounds = 3

In [117]:
ruleset =  GraphColoringRuleset("Graph Coloring Ruleset", initial_state, bounds)
strategy = GraphColoringStrategy("Random Strategy for Graph Coloring")

p1 = Player("player1", strategy)
p2 = Player("player2", strategy)

board = Board(ruleset.initial_state, ruleset.bounds)

ref = Referee(board, ruleset)

game = Game(ref, board, [p1, p2])

board = game.play(True)

player1 -> {'vertex': 0, 'color': 1}
player2 -> {'vertex': 1, 'color': 2}
player1 -> {'vertex': 2, 'color': 1}
player2 -> None
Winner: player1


In [119]:
board.state

[{'color': 1, 'adj': [1, 3]},
 {'color': 2, 'adj': [0, 2, 3]},
 {'color': 1, 'adj': [1]},
 {'color': 0, 'adj': [0, 1]}]